## Medical dataset

NOTE: UK Biobank is not a public dataset - access can be requested https://www.ukbiobank.ac.uk/

Prerequisites:
- enc_ukb file from UK Biobank with the required variables (listed in `inputs/covariates.txt`) and the software programs for working with this type of file
- UK Biobank Primary Care Linked Data for the gp_clinical and gp_scripts tables. See UK Biobank Resource 591 for further details

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import numpy as np
from scipy import stats

PATH_TO_ROOT = '' # TODO replace
PATH_TO_UKBB_DATA = '' # TODO replace
UKBB_APPLICATION_NUMBER = '' # TODO replace
OUTPATH = f'{PATH_TO_ROOT}/data/medical/data/raw'
DATAPATH = 'data/ukb{UKBB_APPLICATION_NUMBER}.csv'
fields_df = pd.read_csv('inputs/covariates.txt')
covariate_fields_df = fields_df[fields_df['Variable']=='covariate']
clinical_fields_df = fields_df[fields_df['Variable']=='clinical']
diagnosis_fields_df = fields_df[fields_df['Variable']=='diagnosis']
tre_df = pd.read_csv('inputs/treatment.txt')

In [ ]:
# connection to db
DBPATH = f'{PATH_TO_UKBB_DATA}/data/raw/ehr/ehr.db'
conn = sqlite3.connect(DBPATH)
conn.text_factory = lambda b: b.decode(errors = 'ignore')

In [ ]:
df = pd.read_csv(DATAPATH)

In [ ]:
# util to remove outlier values in a column of a pandas dataframe
outlier_df = lambda df, col: df[(np.abs(stats.zscore(df[col])) < 3)]

In [ ]:
# get diagnosis variables (aggregate from several ICD codes)
DVT = ['132198-0.0','131400-0.0']
AAA = ['131394-0.0','131382-0.0']
Stroke = ['131378-0.0','131374-0.0','131372-0.0','131366-0.0','131058-0.0']
CAD = ['131306-0.0','131304-0.0','131302-0.0','131300-0.0','131298-0.0','131296-0.0']

outcomes = {'DVT': DVT, 'AAA': AAA, 'Stroke': Stroke, 'CAD': CAD}

for outcome in outcomes:
    for code in outcomes[outcome]:
        df[code] = pd.to_datetime(df[code])
    df[outcome] = df[outcomes[outcome]].min(axis=1)

df['Any_Outcome'] = df[['DVT','AAA','Stroke','CAD']].min(axis=1)


In [ ]:
# get covariates (age, sex)
data = df[['eid','130708-0.0','31-0.0','34-0.0','AAA','DVT','Stroke','CAD','Any_Outcome']].rename(columns={'31-0.0':'Sex','34-0.0':'Year of birth'})
data = data.dropna(subset=['130708-0.0']).rename(columns={'130708-0.0':'T2D'})
patient_list = data['eid'].tolist()
data.to_csv('{}/patient_covariates.csv'.format(OUTPATH), index=None)

In [ ]:
# get treatments (from prescriptions table)
table='gp_scripts'

read2_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='read_2']['Code']])
bnf_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='bnf_code']['Code']])
dmd_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='dmd_code']['Code']])

command = 'SELECT * FROM {} WHERE (read_2 IN ({}) OR bnf_code IN ({}) OR dmd_code IN ({}))'.format(table, read2_codes, bnf_codes, dmd_codes)

data = pd.read_sql_query(command, conn)

# map the codes to the drug label
code_map = dict(zip(tre_df['Code'],tre_df['Label'])) # use read 2 codes because drug name is blank
t1 = data[data['read_2']!='']
t1['treatment'] = t1['read_2'].map(code_map)
t2 = data[data['read_2']=='']
code_map = dict(zip(tre_df['Description'],tre_df['Label'])) # use description because other drugs had same codes
t2['treatment'] = t2['drug_name'].map(code_map)

data = pd.concat([t1,t2])

treatments = {'atorvastatin':'atorvastatin',
              'lipitor':'atorvastatin', 
              'fluvastatin':'fluvastatin', 
              'lescol':'fluvastatin', 
              'lovastatin':'lovastatin',
              'altoprev':'lovastatin', 
              'pitavastatin':'pitavastatin', 
              'livalo':'pitavastatin', 
              'zypitamag':'pitavastatin', 
              'pravastatin':'pravastatin',
              'pravachol':'pravastatin', 
              'rosuvastatin':'rosuvastatin',
              'crestor':'rosuvastatin',
              'ezallor':'rosuvastatin', 
              'simvastatin':'simvastatin', 
              'zocor':'simvastatin'}

t1 = data[~data['treatment'].isna()]
t2 = data[data['treatment'].isna()]

t2['treatment'] = t2['drug_name'].apply(lambda x: next((v for k, v in treatments.items() if k in x.lower()), None))
print("{} rows cannot be identified".format(len(t2[t2['treatment'].isna()]))) 
t2 = t2[~t2['treatment'].isna()]
data = pd.concat([t1,t2])

# data cleaning
data = data[['eid','issue_date','treatment']]
data = data[data['eid'].astype('int').isin(patient_list)]
data = data.dropna()
data = data[~(data['issue_date']=='')]
data['issue_date'] = pd.to_datetime(data['issue_date'], format='%d/%m/%Y')
data.rename(columns={'issue_date':'event_dt'}, inplace=True)

data.to_csv('{}/treatment.csv'.format(OUTPATH), index=None)

In [ ]:
# get biomarkers
table='gp_clinical'
read2_codes = ', '.join(["'{}'".format(x) for x in clinical_fields_df[clinical_fields_df['Type']=='read2']['Code']])
read3_codes = ', '.join(["'{}'".format(x) for x in clinical_fields_df[clinical_fields_df['Type']=='read3']['Code']])

if len(read2_codes)>0 and len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE (read_2 IN ({}) OR read_3 IN ({}))'.format(table, read2_codes, read3_codes)
elif len(read2_codes)>0:
    command = 'SELECT * FROM {} WHERE read_2 IN ({})'.format(table, read2_codes)
elif len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE read_3 IN ({})'.format(table, read3_codes)
    
data = pd.read_sql_query(command, conn)

# data cleaning
code_map = dict(zip(clinical_fields_df['Code'],clinical_fields_df['Description']))
data['read_3'] = data['read_3'].map(code_map)
data['read_2'] = data['read_2'].map(code_map)
data['read_3'] = data['read_3'].fillna(data['read_2'])
data['value'] = pd.to_numeric(data['value1'], errors='coerce')
data = data[['eid','event_dt','read_3','value']]
data = data[data['eid'].astype('int').isin(patient_list)]
data = data.dropna()
data = data[~(data['event_dt']=='')]
data['event_dt'] = pd.to_datetime(data['event_dt'], format='%d/%m/%Y')
data.rename(columns={'read_3':'variable'}, inplace=True)

# remove outliers
data = data[data['value']>0] 
final_data = pd.DataFrame()

for variable in data['variable'].unique():
    final_data = pd.concat([final_data,outlier_df(data[data['variable']==variable], 'value')])

final_data.to_csv('{}/covariate_clinical.csv'.format(OUTPATH), index=None)


Assemble the different data sources into a dataset

In [ ]:
df = pd.read_csv('{}/patient_covariates.csv'.format(OUTPATH))

# convert all datetime columns to years
for col in ['T2D','AAA','DVT','Stroke','CAD']:
    df[col] = pd.to_datetime(df[col]).dt.year

df['Age_at_T2D_diagnosis'] = df['T2D'] - df['Year of birth']
# remove patients diagnosed before birth - indicates an error in the record
df = df[df['Age_at_T2D_diagnosis']>=0]

INDEX_YEAR = 2000 # only consider patients with a diagnosis before the index year
FINAL_YEAR = 2020 # the final year to consider patients for
num_samples_per_task = 12 # fix across patients

data = df[df['T2D']<INDEX_YEAR]

In [ ]:
from tqdm import tqdm

patient_list = data['eid'].tolist()

df_clin = pd.read_csv('{}/covariate_clinical.csv'.format(OUTPATH))
df_clin['type'] = 'biomarker'
df_tre = pd.read_csv('{}/treatment.csv'.format(OUTPATH))
df_tre.rename(columns={'treatment':'variable'}, inplace=True)
df_tre['value'] = 1
df_tre['type'] = 'intervention'
intervention_variables = df_tre['variable'].unique().tolist()
df_clin = pd.concat([df_clin, df_tre])

df_clin['event_dt'] = pd.to_datetime(df_clin['event_dt']).dt.year
df_clin = df_clin[(df_clin['event_dt']>=INDEX_YEAR)&(df_clin['event_dt']<=FINAL_YEAR)]

final_df = pd.DataFrame()

num_Y = {}

# only consider patients with at least 1 patient record
for patient in tqdm(patient_list):

    tmp = df_clin[df_clin['eid']==patient]

    if 'Cholesterol' in tmp['variable'].tolist():

        tmp = tmp.groupby(['event_dt','variable']).mean().reset_index().pivot(index='event_dt',columns='variable',values='value').reset_index()

        # only backfill previous years (e.g., patient may have died before FINAL_YEAR)
        tmp = pd.merge(pd.DataFrame({'event_dt':range(INDEX_YEAR,tmp['event_dt'].max()+1)}), tmp, on='event_dt', how='left')

        num_Y[patient] = len(tmp[~tmp['Cholesterol'].isna()])
        
        # fill missing intervention variables with 0 (to indicate that no intervention occurred)
        tmp_interv = list(set(tmp.keys()).intersection(set(intervention_variables)))
        if len(tmp_interv)>0:
            tmp[tmp_interv] = tmp[tmp_interv].fillna(0)

        # impute the other variables using the most recent value
        tmp = tmp.ffill().bfill()
        tmp.insert(0, 'eid', patient)
        final_df = pd.concat([final_df, tmp])


In [ ]:
# data cleaning
final_df[intervention_variables] = final_df[intervention_variables].fillna(0)
df = final_df.dropna()

tmp = df.groupby('eid').count()
tmp['num_Y'] = tmp.index.map(num_Y)
num_Y_per_task = num_samples_per_task
patient_list = tmp[(tmp['event_dt']>=num_samples_per_task) & (tmp['num_Y']>=num_samples_per_task)].index.tolist()

df = df[df['eid'].isin(patient_list)]

label_df = pd.read_csv('{}/patient_covariates.csv'.format(OUTPATH))

for col in ['T2D','AAA','DVT','Stroke','CAD','Any_Outcome']:
    label_df[col] = pd.to_datetime(label_df[col]).dt.year

label_df = label_df[label_df['eid'].isin(patient_list)]
df = pd.merge(df, label_df, on='eid', how='left')

df['Age'] = df['event_dt'] - df['Year of birth']
df['Age at T2D diagnosis'] = df['T2D'] - df['Year of birth']
df = df.drop(columns=['T2D','Year of birth'])

for col in ['AAA','DVT','Stroke','CAD','Any_Outcome']:
    # indicator = 1 if patient has any history with that disease
    df[col] = (df[col] <= df['event_dt']).astype(int)

df = df[df['event_dt']<=FINAL_YEAR]

tmp = df.groupby('eid').count()
patient_list = tmp[tmp['event_dt']>=num_samples_per_task].index.tolist()

# only keep patients with min number of data samples
df = df[(df['eid'].isin(patient_list))]

# fix issue where some Creatinine values are given in wrong unit
df['Creatinine'] = df['Creatinine'].apply(lambda x : x/1000 if x>1000 else x)

df = df[['eid','Cholesterol','BMI','DBP','SBP','atorvastatin', 'simvastatin','rosuvastatin', 'pravastatin', 'fluvastatin','Sex','AAA','DVT','Stroke','CAD','Age','Age at T2D diagnosis']]

# combine infrequent statins into one variable
df['other_statin'] = (df['rosuvastatin'] + df['pravastatin'] + df['fluvastatin'] >0).astype(int)
df = df.drop(columns=['rosuvastatin', 'pravastatin', 'fluvastatin'])

# convert the task column to indices
task_map = dict(zip(df['eid'].unique(), range(len(df['eid'].unique()))))
df['task'] = df['eid'].map(task_map)
df = df.drop(columns=['eid'])

# rename the feature columns
df = df.rename(columns={'Cholesterol':'Y'})
df = df.rename(columns={feature:'X_{}'.format(feature) for feature in df.keys() if feature not in ['Y','task']})

# same number of samples per patient
final_df = pd.DataFrame()
for patient in df['task'].unique():
    tmp = df[df['task']==patient][0:num_samples_per_task]
    final_df = pd.concat([final_df, tmp])

In [ ]:
# check for null values
final_df.info()

In [ ]:
# analytics
final_df.drop(columns='task').mean()

In [ ]:
import sys
sys.path.insert(1, '../')
from utils import get_train_val_test_data

NUM_DATASETS = 6
DATASET_NAME = 'medical'
INTERVENTIONS = ['X_atorvastatin', 'X_simvastatin', 'X_other_statin']

full_datasets, full_interv_masks = get_train_val_test_data(final_df, NUM_DATASETS, INTERVENTIONS)

for dataset in range(NUM_DATASETS):
    tmp = full_datasets[dataset]
    tmp.to_csv(f'data/processed/{DATASET_NAME}_dataset{dataset}.csv', index=None)
    full_interv_masks[dataset].to_csv(f'data/processed/{DATASET_NAME}_dataset{dataset}_mask.csv', index=None)